# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews



# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Boende,-0.2167,20.8667,71.26,97,100,4.03,CD,1669062227
1,1,Yellowknife,62.4560,-114.3525,28.92,93,75,4.61,CA,1669062264
2,2,Zemio,5.0314,25.1361,74.80,45,80,1.25,CF,1669062496
3,3,Klaebu,63.2976,10.4826,28.63,74,100,4.79,NO,1669062497
4,4,Kapaa,22.0752,-159.3190,82.38,70,0,18.41,US,1669062498


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
locations = city_data_df[["Lat","Lng"]].astype(float)
humidity = city_data_df["Humidity"].astype(float)

map_plot_1 = city_data_df.hvplot.points(
    locations,
    size=humidity,
    geo=True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500    
)

# Display the map
map_plot_1

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Lat'] < 0) & (city_data_df['Wind Speed'] < 15) &
                            (city_data_df['Max Temp'] > 80) & (city_data_df['Max Temp'] < 90),:]

# Drop any rows with null values
clean_weather = ideal_weather_df.dropna(how='any')

# Display sample data
clean_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Pitimbu,-7.4706,-34.8086,81.19,72,73,11.74,BR,1669052993
60,60,Sorong,-0.8833,131.2500,80.87,82,96,9.37,ID,1669053142
73,73,Alyangula,-13.8483,136.4192,82.99,71,83,6.62,AU,1669053160
84,84,Hithadhoo,-0.6000,73.0833,80.71,78,64,10.56,MV,1669053023
182,182,Puerto Leguizamo,-0.1934,-74.7819,85.69,63,99,3.91,CO,1669053250


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,Pitimbu,BR,-7.4706,-34.8086,72,
60,Sorong,ID,-0.8833,131.2500,82,
73,Alyangula,AU,-13.8483,136.4192,71,
84,Hithadhoo,MV,-0.6000,73.0833,78,
182,Puerto Leguizamo,CO,-0.1934,-74.7819,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000
params = {
  "radius": radius,
    "type": "lodging",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Pitimbu - nearest hotel: No hotel found
Sorong - nearest hotel: No hotel found
Alyangula - nearest hotel: No hotel found
Hithadhoo - nearest hotel: No hotel found
Puerto Leguizamo - nearest hotel: No hotel found
Sao Joao Del Rei - nearest hotel: No hotel found
Sola - nearest hotel: No hotel found
Poum - nearest hotel: No hotel found
Cabedelo - nearest hotel: No hotel found
Lorengau - nearest hotel: No hotel found
Maragogi - nearest hotel: No hotel found
Cavalcante - nearest hotel: No hotel found
Vallenar - nearest hotel: No hotel found
Koungou - nearest hotel: No hotel found
Malindi - nearest hotel: No hotel found
Kavieng - nearest hotel: No hotel found
La Ligua - nearest hotel: No hotel found
Rio Grande - nearest hotel: No hotel found
Carmo Do Rio Verde - nearest hotel: No hotel found
Sao Felix Do Xingu - nearest hotel: No hotel found
Tual - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
3,Pitimbu,BR,-7.4706,-34.8086,72,No hotel found
60,Sorong,ID,-0.8833,131.2500,82,No hotel found
73,Alyangula,AU,-13.8483,136.4192,71,No hotel found
84,Hithadhoo,MV,-0.6000,73.0833,78,No hotel found
182,Puerto Leguizamo,CO,-0.1934,-74.7819,63,No hotel found
185,Sao Joao Del Rei,BR,-21.1356,-44.2617,38,No hotel found
253,Sola,VU,-13.8833,167.5500,74,No hotel found
262,Poum,NC,-20.2333,164.0167,81,No hotel found
282,Cabedelo,BR,-6.9811,-34.8339,71,No hotel found
305,Lorengau,PG,-2.0226,147.2712,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)